# Words as Predictors Modelling

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import(
 SGDRegressor,
 SGDClassifier,
 LogisticRegression,
 LogisticRegressionCV,
 LinearRegression,
)

from sklearn.ensemble import(
    AdaBoostRegressor,
    BaggingRegressor,
    AdaBoostClassifier,
    BaggingClassifier,
)

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import (
    train_test_split,
    KFold,
    StratifiedKFold,
    RepeatedStratifiedKFold,
    cross_val_score,
    permutation_test_score,
    RepeatedKFold
)
from sklearn.inspection import permutation_importance

from sklearn.model_selection import GridSearchCV
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.metrics import (
    roc_auc_score,
    f1_score
)
from scipy.stats import sem
import os

## Loading Datasets

In [5]:
#loading the 8 datasets into a list

# assign path
path, dirs, files = next(os.walk("Data/words_as_predictors"))
file_count = len(files)
# create empty list
df_list = []
 
# append datasets to the list
for i in range(file_count):
    #reading the data
    temp_df = pd.read_csv("Data/words_as_predictors/"+files[i])
    #reorder category levels
    word_names = list(temp_df.iloc[:, 1:61].columns)
    target_names = list(temp_df.iloc[:, 65:67])
    temp_df = temp_df.apply(lambda x: x.astype('category') if x.name in word_names else x)
    temp_df = temp_df.apply(lambda x: x.astype('category') if x.name in target_names else x)
    temp_df = temp_df.transform(lambda x: x.cat.set_categories([-1, 1]) if x.name in word_names else x)
    temp_df = temp_df.transform(lambda x: x.cat.set_categories([0,1]) if x.name in target_names else x)

    #appending to the list
    df_list.append(temp_df)
     
# display datasets
for dataset in df_list:
    display(dataset)


,ID,helped,advantage,applause,awe,banjo,banker,belief,bicycle,blue,...,F1,F2,F1_transformed,F2_transformed,Depressed_F1,Anxiety_F2,PHQ9,GAD7,Depressed,Anxiety
0,R_1BOWltIDcyNGKoH,-1,1,1,1,1,1,1,1,1,...,0.887753,1.154815,0.995467,1.162114,1,1,17,15,1,1
1,R_2bVMkt1hhhln2jP,-1,-1,-1,-1,1,1,1,-1,-1,...,-1.296660,-0.820741,-2.038769,-1.048750,0,0,0,4,0,0
2,R_3nCM4LsfCsIjOo7,-1,-1,-1,-1,1,-1,-1,1,-1,...,0.927729,-1.227558,1.017550,-1.838252,1,0,11,2,1,0
3,R_wRBZJ21QBhdo7p7,-1,-1,1,1,-1,1,1,-1,-1,...,0.135663,0.729673,0.341253,0.859894,0,1,11,11,1,1
4,R_1JVecwADD1hEEhd,1,1,1,1,1,1,1,1,1,...,0.265664,-0.662713,0.493820,-0.713643,0,0,8,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,R_pDZ2ZuUBqQRdMC5,-1,-1,-1,-1,1,1,-1,1,1,...,0.500659,-0.144696,0.700827,0.063665,1,0,10,5,1,0
358,R_2y25DrzlaPhcmtt,-1,-1,-1,1,1,1,-1,1,1,...,-0.495961,2.167786,-0.422681,1.964812,0,1,6,8,0,0
359,R_27ecbNVYxed0jd9,-1,-1,-1,-1,1,-1,-1,-1,1,...,0.029661,-0.151589,0.212413,0.057833,0,0,5,1,0,0
360,R_2fBwux8TV0UvpVo,1,1,1,1,-1,1,-1,1,1,...,1.316136,1.770656,1.306331,1.635912,1,1,18,16,1,1


,ID,aggressive,angry,barbecue,bell,book,business,cab,cabbage,cafeteria,...,F1,F2,F1_transformed,F2_transformed,Depressed_F1,Anxiety_F2,PHQ9,GAD7,Depressed,Anxiety
0,R_1Dr6S3X8XV7aNWq,-1,-1,-1,1,-1,1,1,-1,1,...,-0.495368,-0.397361,-0.419046,-0.251716,0,0,5,4,0,0
1,R_2rwqMlB1yZpbsmB,-1,-1,-1,1,-1,-1,1,1,-1,...,0.502047,-0.616587,0.701891,-0.625182,1,0,7,3,0,0
2,R_xfpq3YIIpCmr0hX,1,1,-1,1,1,1,-1,1,1,...,0.807825,1.743641,0.947474,1.623361,1,1,19,20,1,1
3,R_129IACvxn4XnW9u,-1,-1,-1,-1,-1,1,1,1,1,...,0.507370,-0.879450,0.706153,-1.176946,1,0,7,2,0,0
4,R_2pJe7TuHyUVWLEj,1,1,1,-1,-1,-1,-1,-1,1,...,-0.777101,-0.862499,-0.806352,-1.135567,0,0,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,R_2wspAAuPbm29eNA,1,-1,1,1,-1,-1,-1,-1,-1,...,0.128452,-0.949058,0.329815,-1.326107,0,0,5,3,0,0
374,R_3rHQ2bcH0LvJ75M,1,1,1,1,1,1,1,1,1,...,-0.380127,-0.621544,-0.278420,-0.638382,0,0,2,0,0,0
375,R_1fdWU7E9pyDH5Zf,1,1,-1,1,1,-1,1,1,1,...,0.469524,-1.046316,0.664085,-1.523086,1,0,10,2,1,0
376,R_30q0onT1a0cxvAW,-1,1,1,1,-1,1,1,1,1,...,0.079551,0.379870,0.276691,0.563650,0,0,7,6,0,0


,ID,activist,ambulance,avalanche,bonfire,broccoli,businessman,camel,camera,cherry,...,F1,F2,F1_transformed,F2_transformed,Depressed_F1,Anxiety_F2,PHQ9,GAD7,Depressed,Anxiety
0,R_1FbPQrke1paVybl,-1,-1,1,-1,-1,-1,-1,1,-1,...,-0.217606,-0.801512,-0.082011,-1.025960,0,0,6,3,0,0
1,R_1LCkV0plBScTIxu,1,1,-1,-1,1,1,1,1,1,...,-0.490839,-0.936651,-0.415417,-1.292786,0,0,4,3,0,0
2,R_2Tz76JQL2jHYPp6,1,1,-1,-1,-1,-1,-1,-1,1,...,-0.748475,0.882903,-0.767250,0.976536,0,1,2,7,0,0
3,R_XjQDjJtVx656vfz,1,1,-1,1,-1,1,1,1,-1,...,-0.229845,-0.536828,-0.091193,-0.455653,0,0,8,5,0,0
4,R_1qffAI4F7bYGKWY,-1,1,1,-1,-1,1,-1,-1,-1,...,0.545842,2.144200,0.725495,1.936931,1,1,15,20,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,R_QbjlBacwbwyxUDT,1,1,1,1,1,1,1,1,1,...,2.217628,1.111713,1.926188,1.117527,1,1,25,14,1,1
407,R_wKJD8UrGX4uQ7w5,-1,-1,1,-1,1,-1,-1,-1,-1,...,0.267599,-0.876053,0.496640,-1.170322,0,0,5,1,0,0
408,R_vq6FulYrVwxzzJT,1,1,1,1,1,-1,-1,-1,1,...,0.647315,-1.113146,0.816176,-1.608023,1,0,9,4,0,0
409,R_3NJ9reKhujZJqso,1,1,-1,-1,1,-1,1,1,-1,...,0.054139,-0.746294,0.239716,-0.900321,0,0,8,7,0,0


,ID,accordion,apology,apricot,bay,beer,cathedral,cheetah,college,corkscrew,...,F1,F2,F1_transformed,F2_transformed,Depressed_F1,Anxiety_F2,PHQ9,GAD7,Depressed,Anxiety
0,R_r2tpTfFLvWCrG0N,1,-1,-1,-1,1,-1,-1,-1,-1,...,-0.140687,-0.290827,0.016218,-0.079507,0,0,7,7,0,0
1,R_1jHvmK6ZxB1pI0v,1,-1,1,1,-1,1,-1,-1,-1,...,-0.747362,-0.572413,-0.760014,-0.532242,0,0,1,1,0,0
2,R_1r2ZiHHbIJG3ZBv,1,1,-1,-1,-1,-1,1,1,-1,...,-0.412833,-0.291041,-0.321922,-0.080342,0,0,3,2,0,0
3,R_2VBHRj1t6GMLotd,1,-1,-1,1,-1,1,1,1,-1,...,-0.350416,1.238361,-0.240572,1.235470,0,1,9,9,0,1
4,R_3KoKrYqRHmPPvAs,1,-1,1,1,1,1,1,1,1,...,0.063914,-0.272590,0.245711,-0.070333,0,0,4,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,R_269bCsGmvYWQzuD,1,1,-1,1,1,1,1,1,1,...,-1.296544,0.642276,-2.032168,0.786367,0,1,2,4,0,0
365,R_2QoebyLn6C3tTg3,1,1,1,1,1,-1,1,-1,1,...,0.111635,-0.086162,0.309684,0.112930,0,0,8,4,0,0
366,R_1FqXAg76AZMtgPc,-1,1,-1,1,-1,-1,1,-1,-1,...,1.294572,0.276627,1.285152,0.474185,1,0,16,12,1,1
367,R_PIryUjJfza0wS2d,1,-1,1,-1,-1,-1,1,-1,1,...,0.236618,-0.376689,0.456576,-0.202216,0,0,9,5,0,0


,ID,damaged,airport,analogy,arm,attribute,audience,bus,car,carriage,...,F1,F2,F1_transformed,F2_transformed,Depressed_F1,Anxiety_F2,PHQ9,GAD7,Depressed,Anxiety
0,R_DO7WBLeWTraBEkh,1,1,1,1,1,-1,1,-1,-1,...,1.614291,1.168208,1.517802,1.171973,1,1,22,11,1,1
1,R_2QJmilO7N7JtBpM,1,1,-1,-1,-1,-1,1,1,1,...,1.890717,0.480741,1.706457,0.651694,1,1,17,18,1,1
2,R_12okEGnKC0uBOVD,1,-1,-1,-1,-1,-1,-1,1,-1,...,-0.571934,-0.336596,-0.515546,-0.157416,0,0,4,5,0,0
3,R_sYltNfFHlnks9VL,-1,-1,-1,-1,-1,-1,1,-1,1,...,0.041351,-0.025115,0.228603,0.167529,0,0,8,7,0,0
4,R_3s0nXgZ0kv9dSZU,1,1,1,1,1,1,1,1,-1,...,-0.877673,-0.295843,-1.035154,-0.085766,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,R_2COZfWakbxxUyKL,-1,-1,1,-1,1,-1,1,-1,1,...,-0.551887,0.487111,-0.488191,0.654782,0,1,3,4,0,0
354,R_1kYTLMEicRgyWfH,-1,1,-1,-1,-1,1,1,1,1,...,-0.514841,0.465141,-0.451965,0.637363,0,0,4,4,0,0
355,R_SVQVqy3JREzdMtj,1,-1,-1,-1,-1,1,1,1,-1,...,-0.498257,-0.327602,-0.428143,-0.128009,0,0,3,2,0,0
356,R_2v8qwwdKqV1xw0t,-1,1,-1,1,1,-1,-1,-1,1,...,-0.826352,-0.528407,-0.926192,-0.445986,0,0,1,0,0,0


,ID,alligator,army,bagpipe,banana,bar,bed,blueberry,boy,bribe,...,F1,F2,F1_transformed,F2_transformed,Depressed_F1,Anxiety_F2,PHQ9,GAD7,Depressed,Anxiety
0,R_2AEbBswtDMx6dmI,1,1,1,-1,1,-1,1,1,-1,...,-0.386227,0.567055,-0.285343,0.721173,0,1,3,3,0,0
1,R_zVkcRzZ7Dg0So5X,1,1,-1,-1,-1,1,-1,-1,-1,...,0.388227,0.121059,0.589181,0.332451,1,0,9,8,0,0
2,R_2tMwJtTOoKQyggX,1,1,1,1,-1,1,-1,-1,-1,...,0.312453,-1.228666,0.539442,-1.847339,0,0,7,1,0,0
3,R_2zuMIprq0a2shga,1,-1,1,-1,1,-1,-1,-1,-1,...,0.322072,-0.724260,0.549086,-0.853891,0,0,7,5,0,0
4,R_3nBKLeznUVme0h3,1,1,1,-1,-1,1,1,1,1,...,-0.185709,-0.703447,-0.046174,-0.809233,0,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,R_1LS9wIU4ihO850B,1,-1,-1,-1,-1,1,-1,1,1,...,-1.144900,0.287288,-1.632750,0.485382,0,0,2,5,0,0
391,R_233NG3QRA7Z1DXa,1,1,1,-1,1,-1,-1,-1,-1,...,0.463222,-0.982002,0.660978,-1.395363,1,0,8,6,0,0
392,R_3Jm8wdSSX4qd8sW,1,1,1,1,-1,-1,-1,1,1,...,1.268266,1.647043,1.260842,1.550167,1,1,21,21,1,1
393,R_1Lb8XHsntEVaPhr,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-0.452972,0.537261,-0.365154,0.700827,0,1,5,7,0,0


,ID,accident,advice,animosity,ant,apartment,artist,author,baseball,big,...,F1,F2,F1_transformed,F2_transformed,Depressed_F1,Anxiety_F2,PHQ9,GAD7,Depressed,Anxiety
0,R_TbISNkHJ4IeaWwF,1,1,-1,1,1,1,1,1,1,...,-0.818020,0.352191,-0.912864,0.543294,0,0,3,7,0,0
1,R_1Ad63eU2Ca1KXLP,1,-1,-1,1,1,1,-1,-1,1,...,-0.666303,-0.514891,-0.653237,-0.423135,0,0,1,0,0,0
2,R_1j7hOSdsSgrjGa7,1,1,-1,-1,-1,1,-1,1,-1,...,-0.828422,-0.629935,-0.963886,-0.656329,0,0,2,3,0,0
3,R_1q4hHLZJoTYJcE4,1,1,-1,-1,-1,-1,-1,1,-1,...,-0.209060,0.174581,-0.075337,0.375843,0,0,6,8,0,0
4,R_tFlteVSB8FKP42J,1,1,1,1,-1,-1,-1,1,1,...,0.528217,0.810712,0.719018,0.919190,1,1,14,12,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,R_XzVZ2LjbSbErdN7,1,-1,1,-1,-1,1,1,1,-1,...,0.609497,-1.068286,0.781843,-1.550167,1,0,7,0,0,0
362,R_1JD9gIWOY7Uu2oX,1,1,1,-1,1,1,1,1,-1,...,-0.403544,-0.849784,-0.312739,-1.105955,0,0,3,1,0,0
363,R_9NqISJzQXhMjhtf,1,1,1,1,-1,-1,1,1,-1,...,-0.359814,1.490234,-0.254292,1.417726,0,1,10,16,1,1
364,R_2Qt8VZbXncWRo8m,1,1,-1,1,-1,1,1,1,-1,...,1.157134,-1.634252,1.178610,-2.434065,1,0,9,1,0,0


,ID,actor,agreement,asparagus,axe,ball,battle,beach,bee,belch,...,F1,F2,F1_transformed,F2_transformed,Depressed_F1,Anxiety_F2,PHQ9,GAD7,Depressed,Anxiety
0,R_1QseubXuFkKITgO,-1,-1,-1,1,1,1,-1,1,1,...,-0.379659,1.174219,-0.277555,1.180278,0,1,6,12,0,1
1,R_55OzDBBbMqXzJwl,1,-1,-1,-1,1,1,-1,1,1,...,-0.983714,-0.016718,-1.284203,0.179771,0,0,0,2,0,0
2,R_51g1JaoyDoCYAaB,1,-1,-1,1,1,1,-1,1,-1,...,-0.045097,0.097808,0.139337,0.310993,0,0,7,6,0,0
3,R_cHYMbmoxKDTLvs5,1,-1,-1,1,1,1,1,1,1,...,-0.280772,-0.998051,-0.160785,-1.422285,0,0,6,5,0,0
4,R_2WO6CXbbg0sRsgl,1,-1,1,-1,1,1,-1,-1,1,...,-0.344585,-0.455657,-0.232874,-0.339491,0,0,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,R_1FPbs2u13Zq0kJI,1,1,-1,-1,-1,1,1,1,1,...,-0.134317,0.053385,0.021209,0.251716,0,0,5,3,0,0
371,R_1dbjo2qaQ3oESqX,1,-1,-1,1,1,1,-1,1,1,...,1.069060,-0.176303,1.120638,0.036185,1,0,12,7,1,0
372,R_1NsmA90QNN4eehi,1,-1,1,1,1,1,1,1,-1,...,-0.970387,0.515200,-1.259003,0.680768,0,1,3,4,0,0
373,R_3RjPDzZ0yehm3oy,1,-1,1,1,-1,-1,-1,-1,1,...,-0.037314,-0.805872,0.153207,-1.031607,0,0,6,4,0,0


## Classification Models

In [3]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

In [4]:
#defining function for permutation test
def permutation_test(features, target): 
    score, permutation_scores, pvalue = permutation_test_score(
                                        best_model,
                                        features,
                                        target,
                                        scoring = 'accuracy',
                                        cv = cv,
                                        n_permutations = 1000,
                                        n_jobs = -1,
                                        random_state=42,                                                                     
                                        verbose = 1)
    return(score, permutation_scores, pvalue)

In [5]:
#defining function for permutation test for feature importance
def permutation_importance_test(features, target):
    r = permutation_importance(best_model,
                               features,
                               target,
                               n_repeats=100,
                               scoring = 'accuracy',
                               n_jobs=-1,
                               random_state=42)
    return(r.importances_mean, r.importances_std, r.importances)

In [6]:
#here we make a holdout set for each group. We stratify for target to keep
#balanced splits

#define list of dataframes
X_train_list = []
X_holdout_list = []
y_train_list = []
y_holdout_list = []

group_nr = 1
#looping over groups
for df in df_list:
    #datasets for predictors and targets
    X = df.iloc[:, 1:61]
    y = df[['Depressed_F1', 'Anxiety_F2']]
    
    #train/holdout-split stratified by target
    X_train, X_holdout, y_train, y_holdout = train_test_split(X, y,
                                                              stratify = y,
                                                              test_size=0.3,
                                                              random_state=42)
    
    #appending to lists
    X_train_list.append(X_train)
    X_holdout_list.append(X_holdout)
    y_train_list.append(y_train)
    y_holdout_list.append(y_holdout)
                
    #next group
    group_nr += 1


In [7]:
#defining empty pandas dataframe
dataframe = pd.DataFrame(columns = ['group',
                                    'target',
                                    'feature_names',
                                    'baseline_score',
                                    'baseline_std_dev',
                                    'baseline_std_error',
                                    'penalty_before_fs',
                                    'regurilization_strength_before_fs',
                                    'l1ratio_before_fs',
                                    'penalty_score_before_fs',
                                    'penalty_std_dev_before_fs',
                                    'fs_score',
                                    'fs_std_dev',
                                    'fs_std_error',
                                    'penalty_after_fs',
                                    'regurilization_strength_after_fs',
                                    'l1ratio_after_fs',
                                    'penalty_score_after_fs',
                                    'penalty_std_dev_after_fs',
                                    'boost_score',
                                    'boost_std_dev',
                                    'boost_std_error',
                                    'bag_score',
                                    'bag_std_dev',
                                    'bag_std_error',
                                    'earlystop_score',
                                    'earlystop_std_dev',
                                    'earlystop_std_error',
                                    'earlystop_regurilization_strength',
                                    'best_model_name',
                                    'best_model_train_accuracy_cv',
                                    'best_model_train_accuracy',
                                    'best_model_train_rocauc',
                                    'best_model_holdout_accuracy',
                                    'best_model_holdout_rocauc',
                                    'best_model_coef',
                                    'best_model_intercept',
                                    'permutation_accuracy_train',
                                    'permutation_accuracy_holdout',
                                    'permutation_scores_train',
                                    'permutation_pvalue_train',
                                    'permutation_scores_holdout',
                                    'permutation_pvalue_holdout',
                                    'FI_mean_train',
                                    'FI_std_train',
                                    'FI_mean_holdout',
                                    'FI_std_holdout'])


#setting up variables for loop and appending
columns = list(dataframe)
data = []
group_nr = 0

#LOOP
#loop over groups
#loop over targets


#looping over each train split group
for X_train in X_train_list:

    print('**********Start Iteration for Group %s**************' %(group_nr+1))

    #Load the holdout set for X and y in the group
    X_holdout = X_holdout_list[group_nr]
    y_holdout = y_holdout_list[group_nr]

    #load target for the group
    y_train = y_train_list[group_nr]

    #looping over targets
    for target in (range(0,2)):
        y_train_target = y_train.iloc[:, target]
        y_name = y_train_target.name
        y_holdout_target = y_holdout.iloc[:, target]

        print('******Start Iteration for Target %s in Group %s********' %(y_name, group_nr+1))


        #-------------------------Baseline Model-----------------------------

        #Creating baseline model without any penalty
        baseline = LogisticRegression(penalty='none',
                                         random_state=42,
                                         n_jobs = -1)

        #Evaluate by cv
        baseline_cv = cross_val_score(baseline,
                                   X_train,
                                   y_train_target,
                                   scoring= 'accuracy',
                                   cv=cv, 
                                   n_jobs=-1,
                                   error_score='raise')

        #Logging Scores
        baseline_score = baseline_cv.mean()
        baseline_std_dev = baseline_cv.std()
        baseline_std_error = sem(baseline_cv)

        print('Baseline Model Completed for group nr %s and target %s' %(group_nr+1, y_name)) 


        #-------------------GRIDSEARCH L1/L2/Elasticnet--------------------------
        #-------------------(Before feature selection)-------------------------
        #-------------------Another way of faature selection------------------------

        # Defining penalties and value of inverse of regurilization strength
        # smaller values of C specify stronger regurilization.
        #also define l1ratio in case elasticnet is chosen 
        penalty = ['l1', 'l2', 'elasticnet', 'none']
        C = np.logspace(-3,3,7)
        l1_ratio = np.linspace(0,1,11)

        #defining a dictionary parameters to search
        parameters = [{"C":np.logspace(-3,3,7), 
                       "penalty": penalty,
                       'l1_ratio': l1_ratio}]

        #making the gridsearch and fitting features and target
        kep = GridSearchCV(
            estimator = LogisticRegression(solver = 'saga',
                                           n_jobs=-1,
                                           random_state=42),
                                           param_grid = parameters,
                                           scoring = 'accuracy',
                                           n_jobs = -1,
                                           cv=cv, 
                                           refit = True).fit(X_train, y_train_target)

        #logging penalty, regurilization strength and best score
        penalty_before_fs = kep.best_estimator_.get_params()['penalty']
        regurilization_strength_before_fs = kep.best_estimator_.get_params()['C']
        l1ratio_before_fs = kep.best_estimator_.get_params()['l1_ratio']
        penalty_score_before_fs = kep.best_score_
        penalty_std_dev_before_fs = kep.cv_results_['std_test_score'][kep.best_index_]
        gridsearch_best_model_before_fs = kep.best_estimator_

        print('Gridsearch before Feature Selection Completed for group nr %s and target %s' %(group_nr+1, y_name)) 

        #-------------------------Feature Selection-----------------------
        #making base estimator logistic regression with no penalty
        base = LogisticRegression(penalty='none',
                                 n_jobs=-1,
                                 random_state=42)

        #making the forward selection with cv
        forward = SequentialFeatureSelector(
        base,
        k_features = 'best',
        cv = cv,
        forward = True, 
        floating = False,
        scoring = 'accuracy', 
        verbose = 2, 
        n_jobs = -1 
        ).fit(X_train, y_train_target)

        #saving forward-output to a dataframe
        le = pd.DataFrame.from_dict(forward.get_metric_dict()).T

        #finding the index of the best combination of features
        max_index = np.argmax(le.avg_score)

        #saving metrics from the best combination
        feature_names = le.iloc[max_index,3]
        fs_score = le.iloc[max_index,2]
        fs_std_dev = le.iloc[max_index, 5]
        fs_std_error = le.iloc[max_index, 6]

        #Choosing best features in X_train and use that in the rest of the loop
        feature_names_list = list(feature_names)
        X_train_after_fs = X_train[[c for c in X_train.columns if c in feature_names_list]]
        #defining holdout sets after feature selection
        X_holdout_after_fs = X_holdout[[c for c in X_holdout.columns if c in feature_names_list]]

        print('Feature Selection Completed for group nr %s and target %s' %(group_nr+1, y_name))

        #------------------GRIDSEARCH L1/L2/Elasticnet-----------------------
        #------------------(After feature selection)-------------------------

        # Defining penalties and value of inverse of regurilization strength
        # smaller values of C specify stronger regurilization.
        #also define l1ratio in case elasticnet is chosen 
        penalty = ['l1', 'l2', 'elasticnet', 'none']
        C = np.logspace(-3,3,7)
        l1_ratio = np.linspace(0,1,11)

        #defining a dictionary parameters to search
        parameters = [{"C":np.logspace(-3,3,7), 
                       "penalty": penalty,
                       'l1_ratio': l1_ratio}]

        #making the gridsearch and fitting features and target
        kep = GridSearchCV(
            estimator = LogisticRegression(solver = 'saga',
                                           n_jobs=-1,
                                           random_state=42),
                                                  param_grid = parameters,
                                                  scoring = 'accuracy',
                                                  n_jobs = -1,
                                                  cv=cv, 
                                                  refit = True).fit(X_train_after_fs, y_train_target)

        #logging penalty, regurilization strength and best score
        penalty_after_fs = kep.best_estimator_.get_params()['penalty']
        regurilization_strength_after_fs = kep.best_estimator_.get_params()['C']
        l1ratio_after_fs = kep.best_estimator_.get_params()['l1_ratio']
        penalty_score_after_fs = kep.best_score_
        penalty_std_dev_after_fs = kep.cv_results_['std_test_score'][kep.best_index_]
        gridsearch_best_model_after_fs = kep.best_estimator_

        print('Gridsearch after Feature Selection Completed for group nr %s and target %s' %(group_nr+1, y_name))

        #-----------------------Adaboost and Bagging------------------------------
        #These three regurilization techniques apply the penalty selected
        #in the gridsearch above

        #AdaBoost
        boost = AdaBoostClassifier(base_estimator = gridsearch_best_model_after_fs,
                                          random_state=42)

        #Bagging
        bag = BaggingClassifier(base_estimator = gridsearch_best_model_after_fs,
                                         random_state=42,
                                         n_jobs = -1)

        #Evaluate Adaboost by cv
        boost_cv = cross_val_score(boost,
                                   X_train_after_fs,
                                   y_train_target,
                                   scoring= 'accuracy',
                                   cv=cv, 
                                   n_jobs=-1,
                                   error_score='raise')

        #Evaluate Bagging by CV
        bag_cv = cross_val_score(bag,
                                   X_train_after_fs,
                                   y_train_target,
                                   scoring= 'accuracy',
                                   cv=cv, 
                                   n_jobs=-1,
                                   error_score='raise')

        #logging scores
        boost_score = boost_cv.mean()
        boost_std_dev = boost_cv.std()
        boost_std_error = sem(boost_cv)

        bag_score = bag_cv.mean()
        bag_std_dev = bag_cv.std()
        bag_std_error = sem(bag_cv)

        print('Adaboost and Bagging Completed for group nr %s and target %s' %(group_nr+1, y_name))


        #----------------------------Earlystopping---------------------

        #Standard Scaling 
        #We fit the scaler only on the training data and standardise
        #both training and test features (not target!) with that scaler
        sc = StandardScaler()
        X_train_after_fs_sc = sc.fit_transform(X_train_after_fs)
        X_holdout_after_fs_sc = sc.transform(X_holdout_after_fs)

        #We cant use the C regurilization parameter from the Gridsearch
        #After feature selection. So we first find the alpha regurilization
        #parameter that is used in the SGDClassifier. The penalty
        # and 1lratio is transfered.

        #defining a dictionary parameters to search
        parameters = [{"alpha":np.logspace(-3,3,7)}]

        #making the gridsearch and fitting features and target
        kep = GridSearchCV(
            estimator = SGDClassifier(loss='log',
                                      penalty=str(penalty_after_fs), 
                                      l1_ratio=l1ratio_after_fs,
                                      n_jobs=-1,
                                      verbose=1,
                                      early_stopping=True,
                                      validation_fraction=0.2),
                                                  param_grid = parameters,
                                                  scoring = 'accuracy',
                                                  n_jobs = -1,
                                                  cv=cv, 
                                                  refit = True).fit(X_train_after_fs_sc, y_train_target)

        #logging alpha
        alpha_SGD = kep.best_estimator_.get_params()['alpha']

        #base model for earlystop using the previous penalty, alpha and l1ratio
        earlystop =  SGDClassifier(loss='log',
                                   penalty=str(penalty_after_fs), 
                                   alpha=alpha_SGD,
                                   l1_ratio=l1ratio_after_fs,
                                   n_jobs=-1,
                                   verbose=1,
                                   early_stopping=True,
                                   validation_fraction=0.2)

        #Evaluate earlystop by cv
        earlystop_cv = cross_val_score(earlystop,
                                   X_train_after_fs_sc,
                                   y_train_target,
                                   scoring= 'accuracy',
                                   cv=cv, 
                                   n_jobs=-1,
                                   error_score='raise')


        earlystop_score = earlystop_cv.mean()
        earlystop_std_dev = earlystop_cv.std()
        earlystop_std_error = sem(earlystop_cv)

        print('Earlystop Completed for group nr %s and target %s' %(group_nr+1, y_name))

        #-----------------------------Model Evaluation---------------------------

        #selecting the best model and its name
        scorings = (baseline_score, penalty_score_before_fs, fs_score,
                  penalty_score_after_fs, boost_score, bag_score, earlystop_score)
        model_names = ('baseline_model', 'penalty_model_before_fs', 'feature_selection_model',
                       'penalty_model_after_fs', 'boost_model', 'bag_model', 'earlystop_model')
        index_max = scorings. index(max(scorings)) 
        best_model_name = model_names[index_max]
        best_model_train_accuracy_cv = scorings[index_max]
        print(best_model_name)

        #Defining the best model
        if index_max == 0:
            best_model = LogisticRegression(penalty='none',
                                         random_state=42,
                                         n_jobs = -1)

        if index_max == 1:
            best_model = gridsearch_best_model_before_fs                                             

        elif index_max == 2:
            best_model = LogisticRegression(penalty='none',
                                            n_jobs=-1,
                                            random_state=42)

        elif index_max == 3:
            best_model = gridsearch_best_model_after_fs

        elif index_max == 4:
            best_model = AdaBoostClassifier(base_estimator = gridsearch_best_model_after_fs,
                                            random_state=42)

        elif index_max == 5:
            best_model = BaggingClassifier(base_estimator = gridsearch_best_model_after_fs,
                                          random_state=42,
                                          n_jobs = -1)
        elif index_max == 6:
            best_model = SGDClassifier(loss='log',
                                       penalty=str(penalty_after_fs), 
                                       alpha=str(alpha_SGD),
                                       l1_ratio=str(l1ratio_after_fs), 
                                       n_jobs=-1,
                                       verbose=1,
                                       early_stopping=True,
                                       validation_fraction=0.2)
            

       #Get fit, score and apply permutation test on
       #the best model based on whether it is before or after feature selection or with scaled features

        #using features before feature selection
        if index_max in (0,1):
            best_model = best_model.fit(X_train, y_train_target)
            best_model_train_accuracy = best_model.score(X_train, y_train_target)
            best_model_holdout_accuracy = best_model.score(X_holdout, y_holdout_target)
            best_model_train_rocauc = roc_auc_score(y_train_target, best_model.predict(X_train))
            best_model_holdout_rocauc = roc_auc_score(y_holdout_target, best_model.predict(X_holdout))
            
            #permutation tests for train and holdout
            permutation_accuracy_train, permutation_scores_train, permutation_pvalue_train = permutation_test(
            X_train, y_train_target)
            permutation_accuracy_holdout, permutation_scores_holdout, permutation_pvalue_holdout = permutation_test(
            X_holdout, y_holdout_target)
            
            #permutation feature importance for train and holdout
            FI_mean_train, FI_std_train, FI_importances_train = permutation_importance_test(
            X_train, y_train_target)
            FI_mean_holdout, FI_std_holdout, FI_importances_holdout = permutation_importance_test(
            X_holdout, y_holdout_target)
            
            #Choosing all faetures in X_train
            feature_names = X_train.columns

            
        #using features after feature selection
        elif index_max in (2,3,4,5):
            best_model = best_model.fit(X_train_after_fs, y_train_target)
            best_model_train_accuracy = best_model.score(X_train_after_fs, y_train_target)
            best_model_holdout_accuracy = best_model.score(X_holdout_after_fs, y_holdout_target)
            best_model_train_rocauc = roc_auc_score(y_train_target, best_model.predict(X_train_after_fs))
            best_model_holdout_rocauc = roc_auc_score(y_holdout_target, best_model.predict(X_holdout_after_fs))
            
            #permutation tests for train and holdout
            permutation_accuracy_train, permutation_scores_train, permutation_pvalue_train = permutation_test(
            X_train_after_fs, y_train_target)
            permutation_accuracy_holdout, permutation_scores_holdout, permutation_pvalue_holdout = permutation_test(
            X_holdout_after_fs, y_holdout_target)
            
            #permutation feature importance for train and holdout
            FI_mean_train, FI_std_train, FI_importances_train = permutation_importance_test(
            X_train_after_fs, y_train_target)
            FI_mean_holdout, FI_std_holdout, FI_importances_holdout = permutation_importance_test(
            X_holdout_after_fs, y_holdout_target)
            

        #using features after feature selection and normalized (used for earlystop)    
        elif index_max == 6:
            best_model = best_model.fit(X_train_after_fs_sc, y_train_target)
            best_model_train_accuracy = best_model.score(X_train_after_fs_sc, y_train_target)
            best_model_holdout_accuracy = best_model.score(X_holdout_after_fs_sc, y_holdout_target)
            best_model_train_rocauc = roc_auc_score(y_train_target, best_model.predict(X_train_after_fs_sc))
            best_model_holdout_rocauc = roc_auc_score(y_holdout_target, best_model.predict(X_holdout_after_fs_sc))    
                
            #permutation tests for train and holdout
            permutation_accuracy_train, permutation_scores_train, permutation_pvalue_train = permutation_test(
            X_train_after_fs_sc, y_train_target)
            permutation_accuracy_holdout, permutation_scores_holdout, permutation_pvalue_holdout = permutation_test(
            X_holdout_after_fs_sc, y_holdout_target)
            
            #permutation feature importance for train and holdout
            FI_mean_train, FI_std_train, FI_importances_train = permutation_importance_test(
            X_train_after_fs_sc, y_train_target)
            FI_mean_holdout, FI_std_holdout, FI_importances_holdout = permutation_importance_test(
            X_holdout_after_fs_sc, y_holdout_target)

        print('Fitting Best Model on Holdout Set Completed for group nr %s and target %s' %(group_nr+1, y_name))
        
        #saving coefficients and intercept
        best_model_coef = best_model.coef_
        best_model_intercept = best_model.intercept_ 

       #--------------------------Saving output-----------------------------

        #defining outputs as list
        output = [group_nr + 1,
                  y_name,
                  feature_names,
                  baseline_score,
                  baseline_std_dev,
                  baseline_std_error,
                  penalty_before_fs,
                  regurilization_strength_before_fs,
                  l1ratio_before_fs,
                  penalty_score_before_fs,
                  penalty_std_dev_before_fs,
                  fs_score,
                  fs_std_dev,
                  fs_std_error,
                  penalty_after_fs,
                  regurilization_strength_after_fs,
                  l1ratio_after_fs,
                  penalty_score_after_fs,
                  penalty_std_dev_after_fs,
                  boost_score,
                  boost_std_dev,
                  boost_std_error,
                  bag_score,
                  bag_std_dev,
                  bag_std_error,
                  earlystop_score,
                  earlystop_std_dev,
                  earlystop_std_error,
                  alpha_SGD,
                  best_model_name,
                  best_model_train_accuracy_cv,
                  best_model_train_accuracy,
                  best_model_train_rocauc,
                  best_model_holdout_accuracy,
                  best_model_holdout_rocauc,
                  best_model_coef,
                  best_model_intercept,
                  permutation_accuracy_train,
                  permutation_accuracy_holdout,
                  permutation_scores_train,
                  permutation_pvalue_train,
                  permutation_scores_holdout,
                  permutation_pvalue_holdout,
                  FI_mean_train,
                  FI_std_train,
                  FI_mean_holdout,
                  FI_std_holdout]

        #turning outputs into zipped
        zipped = zip(columns, output)
        #turning zipped into dictionary
        output_dict = dict(zipped)
        #appending to our data
        data.append(output_dict)

        print(group_nr+1)

    #next group
    group_nr += 1

#finally (phew) appending data to dataframe
dataframe = dataframe.append(data, ignore_index = True)

**********Start Iteration for Group 1**************
******Start Iteration for Target Depressed_F1 in Group 1********
Baseline Model Completed for group nr 1 and target Depressed_F1


C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Gridsearch before Feature Selection Completed for group nr 1 and target Depressed_F1


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   25.4s finished

[2022-10-19 08:01:51] Features: 1/60 -- score: 0.7232941176470586[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:   25.1s finished

[2022-10-19 08:02:16] Features: 2/60 -- score: 0.7232941176470586[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done  58 out of  58 | elapsed:   25.4s finished

[2022-10-19 08:02:42] Features: 3/60 -- score: 0.7232941176470586[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done  57 out of  57 | elapsed:   24.6s finished

[2022-10-19 08:03:07] Features: 4/60 -- s

[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:   11.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:   11.3s finished

[2022-10-19 08:11:21] Features: 33/60 -- score: 0.6703843137254901[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   11.1s finished

[2022-10-19 08:11:32] Features: 34/60 -- score: 0.6711215686274511[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 out of  26 | elapsed:   11.5s finished

[2022-10-19 08:11:44] Features: 35/60 -- score: 0.6655686274509804[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:    8.3s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   11.1s finished

[2022-10-19 08:11:55] Features: 36/60 -- score: 0.662478431372549[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 con

...

C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\ah140\AppData\Roaming\Python\Py

C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\ah140\AppData\Roaming\Python\Py

C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\ah140\AppData\Roaming\Python\Py

C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\ah140\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\ah140\AppData\Roaming\Python\Py

Fitting Best Model on Holdout Set Completed for group nr 8 and target Anxiety_F2
8


In [47]:
#dataframe.to_csv('Data/words_as_predictors_performance/leplep4.csv')

In [8]:
dataframe

,group,target,feature_names,baseline_score,baseline_std_dev,baseline_std_error,penalty_before_fs,regurilization_strength_before_fs,l1ratio_before_fs,penalty_score_before_fs,...,permutation_accuracy_train,permutation_accuracy_holdout,permutation_scores_train,permutation_pvalue_train,permutation_scores_holdout,permutation_pvalue_holdout,FI_mean_train,FI_std_train,FI_mean_holdout,FI_std_holdout
0,1,Depressed_F1,"(helped, advantage, applause, awe, banjo, chee...",0.623671,0.055354,0.011299,l2,0.010,0.0,0.724078,...,0.728031,0.667532,"[0.7138196078431371, 0.6877019607843137, 0.705...",0.002997,"[0.6712554112554113, 0.6618181818181819, 0.638...",0.414585,"[0.0019367588932806235, 0.009920948616600785, ...","[0.004275174532154481, 0.004347646421009361, 0...","[-9.174311926608336e-05, 0.003944954128440335,...","[0.006551126838694387, 0.007253510624455761, 0..."
1,1,Anxiety_F2,"(helped, advantage, awe, belief, bicycle, fiel...",0.632408,0.049385,0.010081,elasticnet,0.100,0.8,0.728078,...,0.749427,0.656797,"[0.7248784313725488, 0.7272941176470588, 0.716...",0.000999,"[0.696883116883117, 0.6600865800865802, 0.7116...",0.973027,"[0.02221343873517784, 0.0005928853754940677, 0...","[0.006886550727200682, 0.0027597312339464153, ...","[-0.014587155963302725, -0.008348623853210984,...","[0.00971741109784779, 0.008215492599962917, 0...."
2,2,Depressed_F1,"(aggressive, angry, barbecue, bell, clang, emb...",0.568926,0.042548,0.008685,l1,0.001,0.0,0.678012,...,0.683338,0.605613,"[0.6742089985486212, 0.6612772133526851, 0.650...",0.005994,"[0.6704347826086956, 0.6011857707509881, 0.622...",0.717283,"[0.005530303030303025, 0.002651515151515149, -...","[0.003447594016055377, 0.0027050865259631986, ...","[0.0, 0.00605263157894737, 0.0, 0.003596491228...","[0.0, 0.006984020806794045, 0.0, 0.00431432723..."
3,2,Anxiety_F2,"(aggressive, angry, barbecue, bell, book, chai...",0.618171,0.049696,0.010144,l1,0.001,0.0,0.734833,...,0.743149,0.684348,"[0.7128592162554426, 0.7105950653120465, 0.700...",0.000999,"[0.6700395256916996, 0.6542292490118576, 0.640...",0.122877,"[-0.0025757575757575733, -7.575757575757569e-0...","[0.0025057326376577596, 0.003122646037200747, ...","[0.0, -0.005438596491228087, -0.00429824561403...","[0.0, 0.004257775824390048, 0.0043850876315614..."
4,3,Depressed_F1,"(activist, ambulance, avalanche, camel, cherry...",0.621525,0.064298,0.013125,l1,0.001,0.0,0.689897,...,0.715717,0.570733,"[0.613200241984271, 0.6439322444041138, 0.6592...",0.000999,"[0.6304666666666666, 0.5664000000000001, 0.553...",0.632368,"[0.003972125435540122, -0.0020557491289198183,...","[0.00311725006917702, 0.005309331704121226, 0....","[0.01048387096774202, 0.008145161290322661, 0....","[0.007258064516129056, 0.005287641925092129, 0..."
5,3,Anxiety_F2,"(activist, ambulance, avalanche, bonfire, busi...",0.635668,0.053650,0.010951,l1,0.001,0.0,0.721234,...,0.721924,0.676000,"[0.7079370840895343, 0.7177374470659408, 0.703...",0.011988,"[0.6355333333333334, 0.6760666666666667, 0.671...",0.529471,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0010483870967741926, -0.004193548...","[0.0, 0.0, 0.0027121244710968102, 0.0040290309..."
6,4,Depressed_F1,"(accordion, apology, dictation, flute, lab, red)",0.577526,0.055338,0.011296,l1,0.001,0.0,0.686048,...,0.713122,0.659605,"[0.677556561085973, 0.6767571644042232, 0.6852...",0.000999,"[0.6434782608695652, 0.6828458498023715, 0.646...",0.596404,"[0.01503875968992241, 0.012790697674418544, 0....","[0.008496774716213809, 0.014341085271317822, 0...","[0.019189189189189233, 0.026486486486486528, -...","[0.013462667104542475, 0.019272542394552134, 0..."
7,4,Anxiety_F2,"(accordion, apology, apricot, bay, honey, insu...",0.669774,0.049072,0.010017,l1,0.001,0.0,0.751961,...,0.762881,0.692016,"[0.7379487179487181, 0.7349170437405732, 0.738...",0.000999,"[0.7005533596837945, 0.688300395256917, 0.6796...",0.315684,"[0.002480620155038764, 0.010271317829457383, 0...","[

### Saving Train and Holdout Datasets

In [ ]:
#loading the 8 datasets into a list

# assign path
path, dirs, files = next(os.walk("Data/words_as_predictors"))
file_count = len(files)
# create empty list
df_list = []
 
# append datasets to the list
for i in range(file_count):
    #reading the data
    temp_df = pd.read_csv("Data/words_as_predictors/"+files[i])
    #reorder category levels
    word_names = list(temp_df.iloc[:, 1:61].columns)
    target_names = list(temp_df.iloc[:, 65:67])
    temp_df = temp_df.apply(lambda x: x.astype('category') if x.name in word_names else x)
    temp_df = temp_df.apply(lambda x: x.astype('category') if x.name in target_names else x)
    temp_df = temp_df.transform(lambda x: x.cat.set_categories([-1, 1]) if x.name in word_names else x)
    temp_df = temp_df.transform(lambda x: x.cat.set_categories([0,1]) if x.name in target_names else x)

    #appending to the list
    df_list.append(temp_df)
     
# display datasets
for dataset in df_list:
    display(dataset)


In [18]:
#here we make a holdout set for each group. We stratify for target to keep
#balanced splits

#define list of dataframes
X_train_list = []
X_holdout_list = []
y_train_list = []
y_holdout_list = []

group_nr = 1
#looping over groups
for df in df_list:
    #datasets for predictors and targets
    X = df.iloc[:, 1:61]
    y = df[['Depressed_F1', 'Anxiety_F2']]
    
    #train/holdout-split stratified by target
    X_train, X_holdout, y_train, y_holdout = train_test_split(X, y,
                                                              stratify = y,
                                                              test_size=0.3,
                                                              random_state=42)
    
    #appending to lists
    X_train_list.append(X_train)
    X_holdout_list.append(X_holdout)
    y_train_list.append(y_train)
    y_holdout_list.append(y_holdout)
                
    #next group
    group_nr += 1


    Depressed_F1 Anxiety_F2
90             0          0
215            1          0
120            0          0
318            0          0
361            0          0
..           ...        ...
198            0          0
29             1          0
280            0          0
95             0          0
82             0          0

[109 rows x 2 columns]
    Depressed_F1 Anxiety_F2
88             0          0
135            0          0
354            1          0
5              1          0
106            0          1
..           ...        ...
158            0          0
131            0          0
116            0          0
14             1          1
318            0          1

[114 rows x 2 columns]
    Depressed_F1 Anxiety_F2
157            0          0
356            0          0
146            1          0
380            0          1
258            0          0
..           ...        ...
1              0          0
180            0          1
406            1          1


In [34]:
sets = [y_train_list, y_holdout_list]

c=0
#looping over train and holdout
for set in sets:
    #changing names
    #if set == y_train_list:
    #    set_name = 'y_train_'
    #else:
    #    set_name = 'y_holdout_'
    
    if c == 0:
        set_name = 'y_train_'
    else:
        set_name = 'y_holdo_'
    c+=1
    
    i=0
    #looping over the 8 groups
    for dataframe in set:
        i+=1
        set_name_number = set_name + str(i)
        
        dataframe.to_csv('Data/words_as_predictors_train_holdout/' +
                         set_name_number + '.csv')



In [14]:
for dataframe in set:
    print(set)

[    Depressed_F1 Anxiety_F2
90             0          0
215            1          0
120            0          0
318            0          0
361            0          0
..           ...        ...
198            0          0
29             1          0
280            0          0
95             0          0
82             0          0

[109 rows x 2 columns],     Depressed_F1 Anxiety_F2
88             0          0
135            0          0
354            1          0
5              1          0
106            0          1
..           ...        ...
158            0          0
131            0          0
116            0          0
14             1          1
318            0          1

[114 rows x 2 columns],     Depressed_F1 Anxiety_F2
157            0          0
356            0          0
146            1          0
380            0          1
258            0          0
..           ...        ...
1              0          0
180            0          1
406            1         

[113 rows x 2 columns]]
[    Depressed_F1 Anxiety_F2
90             0          0
215            1          0
120            0          0
318            0          0
361            0          0
..           ...        ...
198            0          0
29             1          0
280            0          0
95             0          0
82             0          0

[109 rows x 2 columns],     Depressed_F1 Anxiety_F2
88             0          0
135            0          0
354            1          0
5              1          0
106            0          1
..           ...        ...
158            0          0
131            0          0
116            0          0
14             1          1
318            0          1

[114 rows x 2 columns],     Depressed_F1 Anxiety_F2
157            0          0
356            0          0
146            1          0
380            0          1
258            0          0
..           ...        ...
1              0          0
180            0          1
4

## Regression Models

In [83]:
#here we make a holdout set for each group. We stratify for target to keep
#balanced splits

#define list of dataframes
X_train_list = []
X_holdout_list = []
y_train_list = []
y_holdout_list = []

group_nr = 1
#looping over groups
for df in df_list:
    #datasets for predictors and targets
    X = df.iloc[:, 1:61]
    y = df[['PHQ9', 'GAD7']]
    
    #train/holdout-split 
    X_train, X_holdout, y_train, y_holdout = train_test_split(X, y,
                                                              test_size=0.3,
                                                              random_state=42)
    
    #appending to lists
    X_train_list.append(X_train)
    X_holdout_list.append(X_holdout)
    y_train_list.append(y_train)
    y_holdout_list.append(y_holdout)
                
    #next group
    group_nr += 1


In [84]:
#defining empty pandas dataframe
dataframe = pd.DataFrame(columns = ['group',
                                    'target',
                                    'feature_names',
                                    'baseline_score',
                                    'baseline_std_dev',
                                    'baseline_std_error',
                                    'penalty_before_fs',
                                    'regurilization_strength_before_fs',
                                    'l1ratio_before_fs',
                                    'penalty_score_before_fs',
                                    'penalty_std_dev_before_fs',
                                    'fs_score',
                                    'fs_std_dev',
                                    'fs_std_error',
                                    'penalty_after_fs',
                                    'regurilization_strength_after_fs',
                                    'l1ratio_after_fs',
                                    'penalty_score_after_fs',
                                    'penalty_std_dev_after_fs',
                                    'boost_score',
                                    'boost_std_dev',
                                    'boost_std_error',
                                    'bag_score',
                                    'bag_std_dev',
                                    'bag_std_error',
                                    'earlystop_score',
                                    'earlystop_std_dev',
                                    'earlystop_std_error',
                                    'best_model_name',
                                    'best_model_train_r2',
                                    'best_model_holdout_r2'])
                                    #'best_model_coef',
                                    #'best_model_intercept'])

#setting up variables for loop and appending
columns = list(dataframe)
data = []
group_nr = 0

#LOOP
#loop over groups
#loop over targets

    
#looping over each train split group
for X_train in X_train_list:

    print('**********Start Iteration for Group %s**************' %(group_nr+1))

    #Load the holdout set for X and y in the group
    X_holdout = X_holdout_list[group_nr]
    y_holdout = y_holdout_list[group_nr]

    #load target for the group
    y_train = y_train_list[group_nr]

    #looping over targets
    for target in (range(0,2)):
        y_train_target = y_train.iloc[:, target]
        y_name = y_train_target.name
        y_holdout_target = y_holdout.iloc[:, target]

        print('******Start Iteration for Target %s in Group %s********' %(y_name, group_nr+1))


        #-------------------------Baseline Model-----------------------------

        #Creating baseline model without any penalty
        baseline = LinearRegression(n_jobs = -1)

        # Defining the k-fold to use for Baseline Model
        cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

        #Evaluate by cv
        baseline_cv = cross_val_score(baseline,
                                   X_train,
                                   y_train_target,
                                   scoring= 'r2',
                                   cv=cv, 
                                   n_jobs=-1,
                                   error_score='raise')

        #Logging Scores
        baseline_score = baseline_cv.mean()
        baseline_std_dev = baseline_cv.std()
        baseline_std_error = sem(baseline_cv)

        print('Baseline Model Completed for group nr %s and target %s' %(group_nr+1, y_name)) 


        #-------------------GRIDSEARCH L1/L2/Elasticnet--------------------------
        #-------------------(Before feature selection)-------------------------
        #-------------------Another way of faature selection------------------------

        #Standard Scaling 
        #We fit the scaler only on the training data and standardise
        #both training and test features (not target!) with that scaler
        sc = StandardScaler()
        X_train_sc = sc.fit_transform(X_train)
        X_holdout_sc = sc.transform(X_holdout)

        # Defining penalties and value of inverse of regurilization strength
        # smaller values of C specify stronger regurilization.
        #also define l1ratio in case elasticnet is chosen 
        penalty = ['l1', 'l2', 'elasticnet', 'none']
        alpha = np.logspace(-3,3,7)
        l1_ratio = np.linspace(0,1,11)

        #defining a dictionary parameters to search
        parameters = [{"alpha":np.logspace(-3,3,7), 
                       "penalty": penalty,
                       'l1_ratio': l1_ratio}]

        #making the gridsearch and fitting features and target
        kep = GridSearchCV(
            estimator = SGDRegressor(random_state=42),
                                           param_grid = parameters,
                                           scoring = 'r2',
                                           n_jobs = -1,
                                           refit = True).fit(X_train_sc, y_train_target)

        #logging penalty, regurilization strength and best score
        penalty_before_fs = kep.best_estimator_.get_params()['penalty']
        regurilization_strength_before_fs = kep.best_estimator_.get_params()['alpha']
        l1ratio_before_fs = kep.best_estimator_.get_params()['l1_ratio']
        penalty_score_before_fs = kep.best_score_
        penalty_std_dev_before_fs = kep.cv_results_['std_test_score'][kep.best_index_]
        gridsearch_best_model_before_fs = kep.best_estimator_

        print('Gridsearch before Feature Selection Completed for group nr %s and target %s' %(group_nr+1, y_name)) 

        #-------------------------Feature Selection-----------------------
        #making base estimator logistic regression with no penalty
        base = LinearRegression(n_jobs = -1)

        #making the forward selection with cv
        forward = SequentialFeatureSelector(
        base,
        k_features = 'best',
        forward = True, 
        floating = False,
        scoring = 'r2', 
        verbose = 2, 
        n_jobs = -1 
        ).fit(X_train, y_train_target)

        #saving forward-output to a dataframe
        le = pd.DataFrame.from_dict(forward.get_metric_dict()).T

        #finding the index of the best combination of features
        max_index = np.argmax(le.avg_score)

        #saving metrics from the best combination
        feature_names = le.iloc[max_index,3]
        fs_score = le.iloc[max_index,2]
        fs_std_dev = le.iloc[max_index, 5]
        fs_std_error = le.iloc[max_index, 6]

        #Choosing best features in X_train and use that in the rest of the loop
        feature_names_list = list(feature_names)
        X_train_after_fs = X_train[[c for c in X_train.columns if c in feature_names_list]]
        #defining holdout sets after feature selection
        X_holdout_after_fs = X_holdout[[c for c in X_holdout.columns if c in feature_names_list]]

        print('Feature Selection Completed for group nr %s and target %s' %(group_nr+1, y_name))

        #------------------GRIDSEARCH L1/L2/Elasticnet-----------------------
        #------------------(After feature selection)-------------------------

        #Standard Scaling 
        #We fit the scaler only on the training data and standardise
        #both training and test features (not target!) with that scaler
        sc = StandardScaler()
        X_train_after_fs_sc = sc.fit_transform(X_train_after_fs)
        X_holdout_after_fs_sc = sc.transform(X_holdout_after_fs)

        # Defining penalties and value of inverse of regurilization strength
        # smaller values of C specify stronger regurilization.
        #also define l1ratio in case elasticnet is chosen 
        penalty = ['l1', 'l2', 'elasticnet', 'none']
        alpha = np.logspace(-3,3,7)
        l1_ratio = np.linspace(0,1,11)

        #defining a dictionary parameters to search
        parameters = [{"alpha":np.logspace(-3,3,7), 
                       "penalty": penalty,
                       'l1_ratio': l1_ratio}]

        #making the gridsearch and fitting features and target
        kep = GridSearchCV(
            estimator = SGDRegressor(random_state=42),
                                                  param_grid = parameters,
                                                  scoring = 'r2',
                                                  n_jobs = -1,
                                                  refit = True).fit(X_train_after_fs_sc, y_train_target)

        #logging penalty, regurilization strength and best score
        penalty_after_fs = kep.best_estimator_.get_params()['penalty']
        regurilization_strength_after_fs = kep.best_estimator_.get_params()['alpha']
        l1ratio_after_fs = kep.best_estimator_.get_params()['l1_ratio']
        penalty_score_after_fs = kep.best_score_
        penalty_std_dev_after_fs = kep.cv_results_['std_test_score'][kep.best_index_]
        gridsearch_best_model_after_fs = kep.best_estimator_

        print('Gridsearch after Feature Selection Completed for group nr %s and target %s' %(group_nr+1, y_name))

        #-----------------------Adaboost and Bagging------------------------------
        #These three regurilization techniques apply the penalty selected
        #in the gridsearch above

        #AdaBoost
        boost = AdaBoostRegressor(base_estimator = gridsearch_best_model_after_fs,
                                          random_state=42)

        #Bagging
        bag = BaggingRegressor(base_estimator = gridsearch_best_model_after_fs,
                                         random_state=42,
                                         n_jobs = -1)


        # Defining the k-fold to use for Adaboost, Bagging
        cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

        #Evaluate Adaboost by cv
        boost_cv = cross_val_score(boost,
                                   X_train_after_fs_sc,
                                   y_train_target,
                                   scoring= 'r2',
                                   cv=cv, 
                                   n_jobs=-1,
                                   error_score='raise')

        #Evaluate Bagging by CV
        bag_cv = cross_val_score(bag,
                                   X_train_after_fs_sc,
                                   y_train_target,
                                   scoring= 'r2',
                                   cv=cv, 
                                   n_jobs=-1,
                                   error_score='raise')

        #logging scores
        boost_score = boost_cv.mean()
        boost_std_dev = boost_cv.std()
        boost_std_error = sem(boost_cv)

        bag_score = bag_cv.mean()
        bag_std_dev = bag_cv.std()
        bag_std_error = sem(bag_cv)

        print('Adaboost and Bagging Completed for group nr %s and target %s' %(group_nr+1, y_name))


        #----------------------------Earlystopping---------------------

        #base model for earlystop using
        earlystop =  SGDRegressor( penalty=str(penalty_after_fs), 
                                   alpha=regurilization_strength_after_fs,
                                   l1_ratio=l1ratio_after_fs,
                                   verbose=1,
                                   early_stopping=True,
                                   validation_fraction=0.2)

        #Evaluate earlystop by cv
        earlystop_cv = cross_val_score(earlystop,
                                   X_train_after_fs_sc,
                                   y_train_target,
                                   scoring= 'r2',
                                   cv=cv, 
                                   n_jobs=-1,
                                   error_score='raise')


        earlystop_score = earlystop_cv.mean()
        earlystop_std_dev = earlystop_cv.std()
        earlystop_std_error = sem(earlystop_cv)

        print('Earlystop Completed for group nr %s and target %s' %(group_nr+1, y_name))

        #-----------------------------Model Evaluation---------------------------

        #selecting the best model and its name
        scorings = (baseline_score, penalty_score_before_fs, fs_score,
                  penalty_score_after_fs, boost_score, bag_score, earlystop_score)
        model_names = ('baseline_model', 'penalty_model_before_fs', 'feature_selection_model',
                       'penalty_model_after_fs', 'boost_model', 'bag_model', 'earlystop_model')
        index_max = scorings. index(max(scorings)) 
        best_model_name = model_names[index_max]
        best_model_train_r2 = scorings[index_max]
        
        print(best_model_name)

        #Defining the best model
        if index_max == 0:
            best_model = LinearRegression(n_jobs = -1)

        if index_max == 1:
            best_model = gridsearchbest_model_before_fs                                             

        elif index_max == 2:
            best_model = LinearRegression(n_jobs = -1)

        elif index_max == 3:
            best_model = gridsearch_best_model_after_fs

        elif index_max == 4:
            best_model = AdaBoostRegressor(base_estimator = gridsearch_best_model_after_fs,
                                            random_state=42)

        elif index_max == 5:
            best_model = BaggingRegressor(base_estimator = gridsearch_best_model_after_fs,
                                          random_state=42,
                                          n_jobs = -1)
        elif index_max == 6:
            best_model = SGDRegressor(penalty=str(penalty_after_fs), 
                                       alpha=regurilization_strength_after_fs,
                                       l1_ratio=l1ratio_after_fs,
                                       verbose=1,
                                       early_stopping=True,
                                       validation_fraction=0.2)

       #Get fit, score, coefficients, intercept and apply permutation test on
       #the best model based on whether it is before or after feature selection or with scaled features
        cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

        #using features before feature selection (non-scaled)
        if index_max == 0:
            best_model = best_model.fit(X_train, y_train_target)
            best_model_holdout_r2 = best_model.score(X_holdout, y_holdout_target)
            #best_model_coef = best_model.coef_
            #best_model_intercept = best_model.intercept_
                      
        #using features before feature selection and scaled
        elif index_max == 1:
            best_model = best_model.fit(X_train_sc, y_train_target)
            best_model_holdout_r2 = best_model.score(X_holdout_sc, y_holdout_target)
            #best_model_coef = best_model.coef_
            #best_model_intercept = best_model.intercept_
            
        #using features after feature selection (non-scaled)
        elif index_max == 2:
            best_model = best_model.fit(X_train_after_fs, y_train_target)
            best_model_holdout_r2 = best_model.score(X_holdout_after_fs, y_holdout_target)
            #best_model_coef = best_model.coef_
            #best_model_intercept = best_model.intercept_
            
        #using features after feature selection and scaled   
        elif index_max in (3,4,5,6):
            best_model = best_model.fit(X_train_after_fs_sc, y_train_target)
            best_model_holdout_r2 = best_model.score(X_holdout_after_fs_sc, y_holdout_target)
            #best_model_coef = best_model.coef_
            #best_model_intercept = best_model.intercept_

        print('Fitting Best Model on Holdout Set Completed for group nr %s and target %s' %(group_nr+1, y_name))


       #--------------------------Saving output-----------------------------

        #defining outputs as list
        output = [group_nr + 1,
                  y_name,
                  feature_names,
                  baseline_score,
                  baseline_std_dev,
                  baseline_std_error,
                  penalty_before_fs,
                  regurilization_strength_before_fs,
                  l1ratio_before_fs,
                  penalty_score_before_fs,
                  penalty_std_dev_before_fs,
                  fs_score,
                  fs_std_dev,
                  fs_std_error,
                  penalty_after_fs,
                  regurilization_strength_after_fs,
                  l1ratio_after_fs,
                  penalty_score_after_fs,
                  penalty_std_dev_after_fs,
                  boost_score,
                  boost_std_dev,
                  boost_std_error,
                  bag_score,
                  bag_std_dev,
                  bag_std_error,
                  earlystop_score,
                  earlystop_std_dev,
                  earlystop_std_error,
                  best_model_name,
                  best_model_train_r2,
                  best_model_holdout_r2]
                  #best_model_coef,
                  #best_model_intercept]

        #turning outputs into zipped
        zipped = zip(columns, output)
        #turning zipped into dictionary
        output_dict = dict(zipped)
        #appending to our data
        data.append(output_dict)

        print(group_nr+1)

    #next group
    group_nr += 1
    

#finally (phew) appending data to dataframe
dataframe = dataframe.append(data, ignore_index = True)

**********Start Iteration for Group 1**************
******Start Iteration for Target PHQ9 in Group 1********
Baseline Model Completed for group nr 1 and target PHQ9
Gridsearch before Feature Selection Completed for group nr 1 and target PHQ9


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished

[2022-10-15 17:38:24] Features: 1/60 -- score: -0.004887255890263375[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  44 out of  59 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.1s finished

[2022-10-15 17:38:24] Features: 2/60 -- score: 0.029069121811238708[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  43 out of  58 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  58 out of  58 | elapsed:    0.1s finished

[2022-10-15 17:38:25] Features: 3/60 -- score: 0.038680800392270974[Paral

[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    0.2s finished

[2022-10-15 17:38:31] Features: 32/60 -- score: 0.025243683001449434[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:    0.2s finished

[2022-10-15 17:38:32] Features: 33/60 -- score: 0.020200133300347554[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    0.2s finished

[2022-10-15 17:38:32] Features: 34/60 -- score: 0.014995540190947686[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 out of  26 | elapsed:    0.1s finished

[2022-10-15 17:38:32] Features: 35/60 -- score: 0.009079216707158232[Parallel(n_jobs=-1)]: Using backend LokyBackend w

Feature Selection Completed for group nr 1 and target PHQ9
Gridsearch after Feature Selection Completed for group nr 1 and target PHQ9
Adaboost and Bagging Completed for group nr 1 and target PHQ9
Earlystop Completed for group nr 1 and target PHQ9
penalty_model_after_fs
Fitting Best Model on Holdout Set Completed for group nr 1 and target PHQ9
1
******Start Iteration for Target GAD7 in Group 1********
Baseline Model Completed for group nr 1 and target GAD7
Gridsearch before Feature Selection Completed for group nr 1 and target GAD7


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  60 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished

[2022-10-15 17:38:42] Features: 1/60 -- score: -0.011890258701085265[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.1s finished

[2022-10-15 17:38:42] Features: 2/60 -- score: 0.014889475791822959[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  43 out of  58 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  58 out of  58 | elapsed:    0.1s finished

[2022-10-15 17:38:42] Features: 3/60 -- score: 0.031044553319230728[Paral

...

[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    0.2s finished

[2022-10-15 17:42:55] Features: 32/60 -- score: 0.11966287621940544[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:    0.2s finished

[2022-10-15 17:42:55] Features: 33/60 -- score: 0.11637411966969158[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    0.2s finished

[2022-10-15 17:42:55] Features: 34/60 -- score: 0.11306679603736391[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 out of  26 | elapsed:    0.2s finished

[2022-10-15 17:42:55] Features: 35/60 -- score: 0.10951995466854354[Parallel(n_jobs=-1)]: Using backend LokyBackend with 

Feature Selection Completed for group nr 8 and target PHQ9
Gridsearch after Feature Selection Completed for group nr 8 and target PHQ9
Adaboost and Bagging Completed for group nr 8 and target PHQ9
Earlystop Completed for group nr 8 and target PHQ9
penalty_model_after_fs
Fitting Best Model on Holdout Set Completed for group nr 8 and target PHQ9
8
******Start Iteration for Target GAD7 in Group 8********
Baseline Model Completed for group nr 8 and target GAD7
Gridsearch before Feature Selection Completed for group nr 8 and target GAD7


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  45 out of  60 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished

[2022-10-15 17:43:06] Features: 1/60 -- score: 0.003966640863704929[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  44 out of  59 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.2s finished

[2022-10-15 17:43:06] Features: 2/60 -- score: 0.020277438409411052[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  58 out of  58 | elapsed:    0.2s finished

[2022-10-15 17:43:07] Features: 3/60 -- score: 0.029635631607646817[Parall

[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    0.2s finished

[2022-10-15 17:43:14] Features: 32/60 -- score: 0.03531547983612908[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:    0.2s finished

[2022-10-15 17:43:15] Features: 33/60 -- score: 0.03055409364495947[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    0.2s finished

[2022-10-15 17:43:15] Features: 34/60 -- score: 0.02633379978498269[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 out of  26 | elapsed:    0.2s finished

[2022-10-15 17:43:15] Features: 35/60 -- score: 0.02176688765384256[Parallel(n_jobs=-1)]: Using backend LokyBackend with 

Feature Selection Completed for group nr 8 and target GAD7
Gridsearch after Feature Selection Completed for group nr 8 and target GAD7
Adaboost and Bagging Completed for group nr 8 and target GAD7
Earlystop Completed for group nr 8 and target GAD7
penalty_model_after_fs
Fitting Best Model on Holdout Set Completed for group nr 8 and target GAD7
8


In [86]:
#dataframe.to_csv('Data/words_as_predictors_performance/regression_raw_scores.csv')

In [85]:
dataframe

,group,target,feature_names,baseline_score,baseline_std_dev,baseline_std_error,penalty_before_fs,regurilization_strength_before_fs,l1ratio_before_fs,penalty_score_before_fs,...,boost_std_error,bag_score,bag_std_dev,bag_std_error,earlystop_score,earlystop_std_dev,earlystop_std_error,best_model_name,best_model_train_r2,best_model_holdout_r2
0,1,PHQ9,"(bicycle, cheese, circus, clue, driver, glass,...",-0.483639,0.256812,0.068636,elasticnet,1.0,0.8,-0.032249,...,0.030660,0.011658,0.095326,0.025477,0.022416,0.105198,0.028115,penalty_model_after_fs,0.086735,-0.158200
1,1,GAD7,"(helped, banjo, bicycle, blue, cheese, circus,...",-0.324462,0.137468,0.036740,elasticnet,1.0,0.3,-0.028208,...,0.019210,0.076051,0.094959,0.025379,0.062861,0.091882,0.024557,penalty_model_after_fs,0.131304,-0.199884
2,2,PHQ9,"(aggressive, church, clang, doctor, embrace, e...",-0.384855,0.171185,0.045751,l2,100.0,0.0,-0.011004,...,0.029370,-0.029545,0.088734,0.023715,-0.061963,0.102518,0.027399,penalty_model_after_fs,0.081890,-0.259939
3,2,GAD7,"(chair, church, embrace, etiquette, excuse, gu...",-0.419189,0.206376,0.055156,l2,1000.0,0.0,-0.010746,...,0.027182,-0.010283,0.111328,0.029754,-0.031897,0.090080,0.024075,penalty_model_after_fs,0.082002,-0.106807
4,3,PHQ9,"(activist, businessman, camera, choir, curse, ...",-0.442998,0.287025,0.076711,l2,10.0,0.0,-0.050389,...,0.050168,-0.041107,0.172052,0.045983,-0.043691,0.111048,0.029679,penalty_model_after_fs,0.028982,-0.128575
5,3,GAD7,"(activist, ambulance, camera, cherry, curse, d...",-0.384474,0.261176,0.069802,elasticnet,1.0,0.2,-0.053492,...,0.018656,0.022423,0.080531,0.021523,0.009751,0.078802,0.021061,penalty_model_after_fs,0.030251,-0.238370
6,4,PHQ9,"(delirium, embassy, insult, lab, leg, man, rio...",-0.438173,0.285771,0.076376,l2,1000.0,0.0,-0.036964,...,0.033683,0.016551,0.114365,0.030565,0.011177,0.093942,0.025107,penalty_model_after_fs,0.044303,-0.035922
7,4,GAD7,"(apology, apricot, cheetah, delirium, goldfish...",-0.430151,0.260509,0.069624,l2,100.0,0.0,-0.024287,...,0.031473,-0.002717,0.129567,0.034628,-0.014092,0.125162,0.033451,penalty_model_after_fs,0.048901,-0.048474
8,5,PHQ9,"(bus, carriage, chestnut, egg, festival, folly...",-0.360415,0.239602,0.064036,l2,100.0,0.0,-0.020656,...,0.053774,0.021910,0.117288,0.031346,0.009690,0.104920,0.028041,penalty_model_after_fs,0.096699,-0.193202
9,5,GAD7,"(car, chestnut, corn, festival, folly, hierarc...",-0.569434,0.298370,0.079743,l2,1000.0,0.0,-0.007717,...,0.072353,-0.024933,0.190035,0.050789,-0.066364,0.162205,0.043351,penalty_model_after_fs,0.064556,0.041962
